节点 Node
关系 Relationship

In [2]:
from py2neo import Node, Relationship,size ,order
#创建节点
a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
r = Relationship(a, 'KNOWS', b)
print(a, b, r)

(alice:Person {name:"Alice"}) (bob:Person {name:"Bob"}) (alice)-[:KNOWS]->(bob)


In [2]:
#增加节点属性
a['age'] = 20
b['age'] = 21
r['time'] = '2017/08/31'
print(a, b, r)

(alice:Person {age:20,name:"Alice"}) (bob:Person {age:21,name:"Bob"}) (alice)-[:KNOWS {time:"2017/08/31"}]->(bob)


In [3]:
#设置属性的默认值
a.setdefault('location', '北京')
print(a)

(alice:Person {age:20,location:"北京",name:"Alice"})


In [4]:
#如果该属性原来存在，则setdefault不会覆盖该属性值
a['location'] = '上海'
a.setdefault('location', '北京')
print(a)

(alice:Person {age:20,location:"上海",name:"Alice"})


In [5]:
#可以通过字典更新属性值
data = {
    'name': 'Amy',
    'age': 21
}
a.update(data)
print(a)

(alice:Person {age:21,location:"上海",name:"Amy"})


In [6]:
#子图，是 Node 和 Relationship 的集合，最简单的构造子图的方式是通过关系运算符
a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
r = Relationship(a, 'KNOWS', b)
s = a | b | r #s是一个子图
print(s)

({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})}, {(alice)-[:KNOWS]->(bob)})


In [7]:
#另外还可以通过 nodes() 和 relationships() 方法获取所有的 Node 和 Relationship
print(s.nodes())
print(s.relationships())

frozenset({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})})
frozenset({(alice)-[:KNOWS]->(bob)})


In [8]:
#还可以利用 & 取 Subgraph 的交集
s1 = a | b | r
s2 = a | b
print(s1 & s2)

({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})}, {})


In [9]:
#还可以分别利用 keys()、labels()、nodes()、relationships()、types() 分别获取 Subgraph 的 Key、Label、Node、Relationship、Relationship Type
s = a | b | r
print(s.keys())
print(s.labels())
print(s.nodes())
print(s.relationships())
print(s.types())

frozenset({'name'})
frozenset({'Person'})
frozenset({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})})
frozenset({(alice)-[:KNOWS]->(bob)})
frozenset({'KNOWS'})


In [10]:
#还可以用 order() 或 size() 方法来获取 Subgraph 的 Node 数量和 Relationship 数量
s = a | b | r
print(order(s))  #节点数量
print(size(s))   #关系数量

2
1


In [11]:
#Walkable 是增加了遍历信息的 Subgraph，我们通过 + 号便可以构建一个 Walkable 对象
a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
c = Node('Person', name='Mike')
ab = Relationship(a, "KNOWS", b)
ac = Relationship(a, "KNOWS", c)
w = ab + Relationship(b, "LIKES", c) + ac
print(w)

(alice)-[:KNOWS]->(bob)-[:LIKES]->(mike)<-[:KNOWS]-(alice)


In [12]:
#可以调用 walk() 方法实现遍历
from py2neo import walk
for item in walk(w):
    print(item)

(alice:Person {name:"Alice"})
(alice)-[:KNOWS]->(bob)
(bob:Person {name:"Bob"})
(bob)-[:LIKES]->(mike)
(mike:Person {name:"Mike"})
(alice)-[:KNOWS]->(mike)
(alice:Person {name:"Alice"})


In [13]:
#可以利用 start_node()、end_node()、nodes()、relationships() 方法来获取起始 Node、终止 Node、所有 Node 和 Relationship
print(w.start_node())
print(w.end_node())
print(w.nodes())
print(w.relationships())

(alice:Person {name:"Alice"})
(alice:Person {name:"Alice"})
((alice:Person {name:"Alice"}), (bob:Person {name:"Bob"}), (mike:Person {name:"Mike"}), (alice:Person {name:"Alice"}))
((alice)-[:KNOWS]->(bob), (bob)-[:LIKES]->(mike), (alice)-[:KNOWS]->(mike))


在 database 模块中包含了和 Neo4j 数据交互的 API，最重要的当属 Graph，它代表了 Neo4j 的图数据库，同时 Graph 也提供了许多方法来操作 Neo4j 数据库。
Graph 在初始化的时候需要传入连接的 URI，初始化参数有 bolt、secure、host、http_port、https_port、bolt_port、user、password，
详情说明可以参考：http://py2neo.org/v3/database.html#py2neo.database.Graph。

In [14]:
#Graph()实例化
from py2neo import Graph
graph = Graph("http://localhost:7474")

In [15]:
#可以利用 create() 方法传入 Subgraph 对象来将关系图添加到数据库中
a = Node('Person', name='Alice')
b = Node('Person', name='Bob')
r = Relationship(a, 'KNOWS', b)
s = a | b | r
graph.create(s)

In [16]:
#可以单独添加Node或Relationship
a = Node('Person', name='Mike')
graph.create(a)
b = Node('Person', name='Jack')
ab = Relationship(a, 'KNOWS', b)
graph.create(ab)

In [17]:
#可以利用 data() 方法来获取查询结果
from py2neo import Graph
data = graph.data('MATCH (p:Person) return p.name ') #通过 CQL 语句实现的查询
print(data)

[{'p.name': 'Alice'}, {'p.name': 'Bob'}, {'p.name': 'Mike'}, {'p.name': 'Jack'}]


In [18]:
#转换为DataFrame
from pandas import DataFrame
df = DataFrame(data)
print(df)

  p.name
0  Alice
1    Bob
2   Mike
3   Jack


In [19]:
a = Node('Person', name='Alice', age=21, location='广州')
b = Node('Person', name='Bob', age=22, location='上海')
c = Node('Person', name='Mike', age=21, location='北京')
r1 = Relationship(a, 'KNOWS', b)
r2 = Relationship(b, 'KNOWS', c)
graph.create(a)
graph.create(r1)
graph.create(r2)

In [20]:
from py2neo import Graph, NodeSelector
#用 NodeSelector 来筛选 age 为 21 的 Person Node
selector = NodeSelector(graph)
persons = selector.select('Person', age=21)
print(list(persons))

[(mike:Person {age:21,location:"北京",name:"Mike"}), (alice:Person {age:21,location:"广州",name:"Alice"})]


In [21]:
#可以使用 where() 进行更复杂的查询，例如查找 name 是 A 开头的 Person Node
persons = selector.select('Person').where('_.name =~ "A.*"')
print(list(persons))

[(alice:Person {name:"Alice"}), (alice:Person {age:21,location:"广州",name:"Alice"})]


In [22]:
#可以利用 match() 或 match_one() 方法对 Relationship 进行查找
relationship = graph.match_one(rel_type='KNOWS')
print(relationship)

(alice)-[:KNOWS]->(bob)


In [23]:
#可以使用 order_by() 进行排序
persons = selector.select('Person').order_by('_.age')
print(list(persons))

[(mike:Person {age:21,location:"北京",name:"Mike"}), (alice:Person {age:21,location:"广州",name:"Alice"}), (bob:Person {age:22,location:"上海",name:"Bob"}), (alice:Person {name:"Alice"}), (bob:Person {name:"Bob"}), (mike:Person {name:"Mike"}), (jack:Person {name:"Jack"})]


In [24]:
#如果要查询单个节点的话，可以使用 first() 方法
person = selector.select('Person').where('_.name =~ "A.*"').first()
print(person)

(alice:Person {name:"Alice"})


In [25]:
#可以通过 run() 方法直接执行 CQL 语句
data = graph.run('MATCH (p:Person) RETURN p LIMIT 5')
print(list(data))

[('p': (alice:Person {name:"Alice"})), ('p': (bob:Person {name:"Bob"})), ('p': (bob:Person {age:22,location:"上海",name:"Bob"})), ('p': (mike:Person {age:21,location:"北京",name:"Mike"})), ('p': (mike:Person {name:"Mike"}))]


OGM 类似于 ORM，意为 Object Graph Mapping，这样可以实现一个对象和 Node 的关联

In [27]:
#可以用它来结合 Graph 查询
from py2neo import Graph
from py2neo.ogm import GraphObject, Property

class Person(GraphObject):
    __primarykey__ = 'name'

    name = Property()
    age = Property()
    location = Property()

person = Person.select(graph).where(age=21).first()
print(person)
print(person.name)
print(person.age)

<Person name='Mike'>
Mike
21


In [28]:
#可以用它动态改变 Node 的属性，例如修改某个 Node 的 age 属性
person = Person.select(graph).where(age=21).first()
print(person.__ogm__.node)
person.age = 22
print(person.__ogm__.node)
graph.push(person)

(mike:Person {age:21,location:"北京",name:"Mike"})
(mike:Person {age:22,location:"北京",name:"Mike"})


In [43]:
from py2neo.ogm import GraphObject, Property, RelatedTo

#也可以通过映射关系进行 Relationship 的调整，例如通过 Relationship 添加一个关联 Node

class Person(GraphObject):
    __primarykey__ = 'name'

    name = Property()
    age = Property()
    location = Property()
    knows = RelatedTo('Person', 'KNOWS')

person = Person.select(graph).where(age=21).first()
print(list(person.knows))
new_person = Person()
new_person.name = 'Durant'
new_person.age = 28
person.knows.add(new_person)
print(list(person.knows))

[<Person name='Bob'>]
[<Person name='Bob'>, <Person name='Durant'>]


这样我们就完成了 Node 和 Relationship 的添加，同时由于设置了 primarykey 为 name，所以不会重复添加。
但是注意此时数据库并没有更新，只是对象更新了，如果要更新到数据库中还需要调用 Graph 对象的 push() 或 pull() 方法，添加如下代码即可：

In [44]:
graph.push(person)

In [45]:
target=Person.select(graph).where(name='Durant').first()
print(list(person.knows))
person.knows.remove(target)  #可以通过remove来删除节点间的关系
print(list(person.knows))

[<Person name='Bob'>, <Person name='Durant'>]
[<Person name='Bob'>]


In [46]:
graph.push(person)

In [47]:
graph.delete(target) #先删除节点上的关系，才能删除节点